# GMMAT Analyses for common variants on the updated AD Data
This notebook documents the population specific GMMAT analyses using the updated AD pheno (https://github.com/gaow/alzheimers-family/blob/master/notebook/20221121_AD_pheno_update.ipynb).

Major updates for the pheno data

* Most of missing data for age has been completed
* missing info for APOE4 updated based on the sequence data
* controls under 60 years of age excluded
* For the European samples (n = 15) age values coded as like 999, 8027 were replaced by the correct age
* unaffected singletons removed 
* PCs recalculated based on the updated pheno 

Pheno data
 > /mnt/mfs/statgen/alzheimers-family/pheno/pheno_updated_20221121/
 
Geno data: WGS data with jointly called EFIGA and NIALOAD data is available here
 > /mnt/mfs/statgen/alzheimers-family/normalized_bed/normalized_merged_autosome.*  
 
 QCed genetic data used for the analyses for African, European and Hispanic
 
 > /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/geno_qced.$i.*

We used GMMAT software included in [our workflow](https://github.com/cumc/bioworkflows/blob/master/GWAS/LMM.ipynb) for the analyses. The docker image that contains all the necessary software installations to run this notebook and workflow can be retrieved by the following command: docker pull statisticalgenetics/lmm:3.0

# Common Variants analyses

## Calculate Kin Matrix

### Generate genoFile with LD pruning as a temporary file to compute GRM

In [ ]:
ml Singularity
for i in African European Hispanic; do
# unrelated individuals
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc \
    --cwd mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --remove_samples /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.$i.filtered.related_id \
    --maf_filter 0.01 \
    --geno_filter 0.1 \
    --mind_filter 0.1 \
    --hwe_filter 5e-08 \
    --name unrelated \
    --container /mnt/mfs/statgen/containers/lmm.sif
# related individuals same set of variants
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc:1 \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --keep_samples /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.$i.filtered.related_id \
    --keep_variants /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.$i.filtered.unrelated.filtered.bim \
    --maf_filter 0 \
    --geno_filter 0.1 \
    --mind_filter 0.1 \
    --hwe_filter 0 \
    --name related \
    --container /mnt/mfs/statgen/containers/lmm.sif 
done

In [ ]:
# merge two data-sets
for i in African European Hispanic; do
    plink --bfile /mnt/vast/hpc/csg/bst2126/notebook_Bale/AD_family/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/normalized_merged_autosome.$i.filtered.related.filtered.extracted \
         --bmerge /mnt/vast/hpc/csg/bst2126/notebook_Bale/AD_family/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/normalized_merged_autosome.$i.filtered.unrelated.filtered.prune.bed \
                  /mnt/vast/hpc/csg/bst2126/notebook_Bale/AD_family/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/normalized_merged_autosome.$i.filtered.unrelated.filtered.prune.bim \
                  /mnt/vast/hpc/csg/bst2126/notebook_Bale/AD_family/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/normalized_merged_autosome.$i.filtered.unrelated.filtered.prune.fam \
        --make-bed --keep-allele-order --out /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.$i
done

In [ ]:
##in order to run gemma, pheno column in fam file can not be missing, so make modification on fam file.
data=read.table('/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.African.fam',header=F)
data[,6]=5
write.table(data,'/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.African.fam',col.names=F,row.names=F,quote=F)

data1=read.table('/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.European.fam',header=F)
data1[,6]=5
write.table(data1,'/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.European.fam',col.names=F,row.names=F,quote=F)

data2=read.table('/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.Hispanic.fam',header=F)
data2[,6]=5
write.table(data2,'/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.Hispanic.fam',col.names=F,row.names=F,quote=F)

### Compute GRM

In [ ]:
ml GEMMA
for i in African European Hispanic; do
gemma \
  -bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned.$i \
  -gk 2 \
  -o geno_pruned_$i \
  -outdir /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship
done

### Null Models

In [ ]:
#MODEL ONE with out APOE4
for i in African European Hispanic; do
module load Singularity
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb null_model \
--cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/null2 \
--bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/geno_qced.$i.fam \
--phenoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/$i.pca.projected.txt \
--grmFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned_$i.sXX.txt \
--phenoCol AD \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--covarCol SEX \
--qCovarCol AGE PC1 PC2 PC3 \
--model_name noAPOE
done

In [ ]:
#MODEL ONE with APOE4
for i in African European Hispanic; do
module load Singularity
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb null_model \
--cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/null2 \
--bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/geno_qced.$i.fam \
--phenoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/$i.pca.projected.txt \
--grmFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/kinship/geno_pruned_$i.sXX.txt \
--phenoCol AD \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--covarCol SEX APOE \
--qCovarCol AGE PC1 PC2 PC3 \
--model_name APOE
done

# Population specific association tests using GMMAT

## Analyses with SEX and AGE as covariates

In [ ]:
#!/bin/sh
#$ -l h_rt=4:00:00
#$ -l h_vmem=40G
#$ -N gmmat_update_noAPOE4
#$ -o /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

for i in African European Hispanic; do
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb GMMAT \
--cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/$i \
--bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/geno_qced.$i.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/null2/geno_qced.$i.$i.pca.projected.noAPOE.rds \
--phenoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/$i.pca.projected.txt \
--formatFile ~/project2022/bioworkflows/GWAS/data/gmmat_template.yml \
--label_annotate SNP \
--phenoCol AD \
--covarMaxLevels 10 \
--numThreads 20 \
--bgenMinMAF 0.01 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100
done

## Analyses with APOE4 and SEX, AGE as covariate

In [ ]:
#!/bin/sh
#$ -l h_rt=23:00:00
#$ -l h_vmem=1000G
#$ -N gmmat_update_APOE4
#$ -o /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH


module load Singularity/3.5.3

for i in African European Hispanic; do
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb GMMAT \
--cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/$i \
--bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/geno_qced.$i.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/null2/geno_qced.$i.$i.pca.projected.APOE.rds \
--phenoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/$i.pca.projected.txt \
--formatFile ~/project2022/bioworkflows/GWAS/data/gmmat_template.yml \
--label_annotate SNP \
--phenoCol AD \
--covarMaxLevels 10 \
--numThreads 20 \
--bgenMinMAF 0.01 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100
done

## Meta-analysis

We used [METAL](https://genome.sph.umich.edu/wiki/METAL_Documentation) to meta-analyze population specific results. 

### with out APOE4

In [ ]:

SCHEME STDERR
MARKER   SNP
WEIGHT   N
ALLELE   A2 A1
EFFECT   BHAT
STDERR   SBHAT
PVAL     PVAL
FREQLABEL AF
GENOMICCONTROL ON
AVERAGEFREQ ON
MINMAXFREQ ON

PROCESS /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/European/geno_qced.European.European.pca.projected.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/Hispanic/geno_qced.Hispanic.Hispanic.pca.projected.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/African/geno_qced.African.African.pca.projected.gmmat.score.txt.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/metal/AD.GMMAT_noAPOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [ ]:
# Reformat the metal result

library(stringr) # to replace strings
library(tidyr) # to get  separate function separate the marker name 
library(data.table)
library(dplyr)

meta_noAPOE4 <- read.table('/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/metal/AD.GMMAT_noAPOE4_META2.TXT', header = T, sep = '\t')
meta_noAPOE4 <- meta_noAPOE4 %>% mutate (SNP = MarkerName)
meta_noAPOE4 <- meta_noAPOE4 %>% mutate(across('MarkerName', str_replace_all, '_', ':'))
meta_noAPOE4 <- meta_noAPOE4 %>% separate(MarkerName, c('CHR','POS','REF', 'ALT'),  sep = ':')
meta_noAPOE4 <- meta_noAPOE4 %>% mutate(across('CHR', str_replace, 'chr', ''))
meta_noAPOE4$POS <-  as.numeric(meta_noAPOE4$POS)
meta_noAPOE4$CHR <-  as.numeric(meta_noAPOE4$CHR)
write.table(meta_noAPOE4,'/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/noAPOE4/metal/meta.noAPOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)

In [5]:
# calculate lamda

lambda <- median(qchisq(meta_noAPOE4$P.value, df=1, lower.tail=FALSE)) / qchisq(0.5, 1)

In [6]:
lambda

[1] 0.9888482

### with APOE4 adjustment

In [2]:
library(dplyr)

In [ ]:
SCHEME STDERR
MARKER   SNP
WEIGHT   N
ALLELE   A2 A1
EFFECT   BHAT
STDERR   SBHAT
PVAL     PVAL
FREQLABEL AF
GENOMICCONTROL ON
AVERAGEFREQ ON
MINMAXFREQ ON

PROCESS /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/European/geno_qced.European.European.pca.projected.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/Hispanic/geno_qced.Hispanic.Hispanic.pca.projected.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/African/geno_qced.African.African.pca.projected.gmmat.score.txt.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/metal/AD.GMMAT_APOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [7]:
# Reformate the metal result and generat manhattan and qq plots

meta_APOE4 <- read.table('/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/metal/AD.GMMAT_APOE4_META2.TXT', header = T, sep = '\t')
meta_APOE4 <- meta_APOE4 %>% mutate (SNP = MarkerName)
meta_APOE4 <- meta_APOE4 %>% mutate(across('MarkerName', str_replace_all, '_', ':'))
meta_APOE4 <- meta_APOE4 %>% separate(MarkerName, c('CHR','POS','REF', 'ALT'),  sep = ':')
meta_APOE4 <- meta_APOE4 %>% mutate(across('CHR', str_replace, 'chr', ''))
meta_APOE4$POS <-  as.numeric(meta_APOE4$POS)
meta_APOE4$CHR <-  as.numeric(meta_APOE4$CHR)
write.table(meta_APOE4,'/mnt/mfs/statgen/alzheimers-family/AD_common_variants/gmmat/results2/APOE4/metal/meta.APOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)

In [ ]:
lambda

[1] 0.9888482